In [16]:
!pip install yfinance ipywidgets numpy matplotlib

import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown, HBox, VBox, Layout, widget, Output, Text
from IPython.display import display, clear_output

In [17]:
plt.rcParams['figure.figsize'] = [12, 6]

DEFAULT_TICKER = ""
DEFAULT_DAYS = 252
DEFAULT_SIMULATIONS = 10
DEFAULT_VOL_ADJUST = 1.0

def get_stock_data(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="5y")
    return hist

def monte_carlo_simulation(ticker, days, n_simulations, vol_adjust):
    data = get_stock_data(ticker)
    if data.empty:
        print(f"No data found for {ticker}")
        return None, None, None

    close_prices = data['Close']
    returns = np.log(1 + close_prices.pct_change())

    mu = returns.mean() * 252
    sigma = returns.std() * np.sqrt(252) * vol_adjust
    last_price = close_prices[-1]

    dt = 1/252
    daily_returns = np.exp((mu - 0.5 * sigma**2) * dt +
                    sigma * np.sqrt(dt) * np.random.normal(size=(days, n_simulations)))

    price_paths = np.zeros_like(daily_returns)
    price_paths[0] = last_price

    for t in range(1, days):
        price_paths[t] = price_paths[t-1] * daily_returns[t]

    return price_paths, mu, sigma

def plot_results(ticker, days, n_simulations, vol_adjust):
    with output:
        clear_output(wait=True)

        paths, mu, sigma = monte_carlo_simulation(ticker, days, n_simulations, vol_adjust)
        if paths is None:
            return

        fig, (ax1, ax2) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [2, 1]})

        ax1.plot(paths, linewidth=1, alpha=0.5)
        ax1.set_title(f'{n_simulations} {ticker} Monte Carlo Simulations\n{days} Trading Days')
        ax1.set_xlabel('Trading Days')
        ax1.set_ylabel('Price ($)')
        ax1.grid(True)

        final_prices = paths[-1]
        ax2.hist(final_prices, bins=30, orientation='horizontal', alpha=0.7)
        ax2.set_title('Final Price Distribution')
        ax2.grid(True)

        plt.tight_layout()

        data = get_stock_data(ticker)
        if data.empty:
            return
        close_prices = data['Close']
        last_price = close_prices[-1]
        mean_price = np.mean(final_prices)
        median_price = np.median(final_prices)
        percentile_5 = np.percentile(final_prices, 5)
        percentile_95 = np.percentile(final_prices, 95)

        print(f"\n Simulation Parameters:")
        print(f"- Ticker: {ticker}")
        print(f"-Days: {days} (≈{days/252:.1f} years)")
        print(f"- Simulations: {n_simulations}")
        print(f"- Volatility Adjustment: {vol_adjust:.2f}x")

        print(f"\n Results:")
        print(f"- Current Price: ${last_price:.2f}")
        print(f"- Expected Price: ${mean_price:.2f}")
        print(f"- Median Price: ${median_price:.2f}")
        print(f"- 5th Percentile: ${percentile_5:.2f}")
        print(f"- 95th Percentile: ${percentile_95:.2f}")

        plt.show()

ticker_input = Text(
    value=DEFAULT_TICKER,
    description='Stock:',
    placeholder='Enter ticker (e.g. AAPL)',
    layout=Layout(width='200px')
)

days_input = IntSlider(
    min=10,
    max=1260,
    step=10,
    value=DEFAULT_DAYS,
    description='Days:',
    layout=Layout(width='300px')
)

sims_input = IntSlider(
    min=10,
    max=1000,
    step=10,
    value=DEFAULT_SIMULATIONS,
    description='Simulations:',
    layout=Layout(width='300px')
)

vol_input = FloatSlider(
    min=0.5,
    max=2.0,
    step=0.1,
    value=DEFAULT_VOL_ADJUST,
    description='Volatility Adj:',
    layout=Layout(width='300px')
)

output = Output()

controls = HBox([ticker_input, days_input, sims_input, vol_input])
display(VBox([controls, output]))

def update_plot(change):
    plot_results(
        ticker_input.value,
        days_input.value,
        sims_input.value,
        vol_input.value
    )

ticker_input.observe(update_plot, names='value')
days_input.observe(update_plot, names='value')
sims_input.observe(update_plot, names='value')
vol_input.observe(update_plot, names='value')

plot_results(DEFAULT_TICKER, DEFAULT_DAYS, DEFAULT_SIMULATIONS, DEFAULT_VOL_ADJUST)